In [133]:
import minsearch
import os
import json
from groq import Groq

Based on the provided text, it can be inferred that the impact of the foreign exchange directive for Ethiopian banks is significant. The directive aims to regulate and standardize the foreign exchange market in Ethiopia, ensuring transparency, stability, and efficiency in foreign exchange transactions.

Some of the impacts of the directive on Ethiopian banks include:

1. Regulation of foreign exchange transactions: The directive establishes regulations for foreign exchange transactions, including reporting requirements, exchange rates, and usage of foreign currencies.
2. Standardization of exchange rates: The directive introduces the concept of the indicative daily exchange rate, which serves as a reference price for foreign exchange transactions. This helps to standardize exchange rates and ensures that banks are using the same rates for their transactions.
3. Increased transparency: The directive requires banks to report their exchange rates and foreign exchange transactions, providi

In [169]:
def process_json_files_and_index(json_folder_path):
    """
    This function reads all JSON files from the specified directory, extracts documents,
    and indexes them using the `minsearch` library.

    Args:
        json_folder_path (str): The directory containing JSON files.

    Returns:
        index: The indexed documents.
    """

    # List to hold all documents from all JSON files
    all_documents = []

    # Loop through each JSON file in the directory
    for json_file in os.listdir(json_folder_path):
        if json_file.endswith(".json"):
            json_file_path = os.path.join(json_folder_path, json_file)
            with open(json_file_path, 'rt', encoding='utf-8') as f_in:
                docs_raw = json.load(f_in)

            # Extract documents from the current JSON file
            documents = []
            for directive_id_dict in docs_raw:
                for doc in directive_id_dict['sections']:
                    # Add additional fields to the doc
                    doc['document_id'] = directive_id_dict['document_id']
                    doc['title'] = directive_id_dict['title']
                    documents.append(doc)

            # Append documents from this file to the global list
            all_documents.extend(documents)

    # Index the documents (from all files)
    index = minsearch.Index(
        text_fields=["section_title", "content", "title"],
        keyword_fields=["document_id"]
    )

    # Index the combined documents
    index.fit(all_documents)
    
    return index


json_folder_path = 'D:\\NBEDirectivesAssist\\ConvertedToJson\\'

In [164]:
def search(query):
    # Boosting specific fields
    boost = {'content': 10.0, 'title': 2, 'section_title': 0.5}

   # Search for the query, filtering by a specific directive
    result = index.search(
    query=q,
    boost_dict=boost,
    num_results=5,
    filter_dict={'title': 'Asset Classification and Provisioning Directive'}
    )
    return result

In [165]:
def build_prompt(query,search_results):
    # Create the prompt for the LLM
    prompt_template = """
You're National Bank of Ethiopia Directives assistant. Answer the QUESTION based on the CONTEXT from the directive document.
Use  facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not contain the answer, Answer from your general knowledge.

QUESTION: {question}
CONTEXT: {context}
     """.strip()

# Generate the context from the search results
    context = ""

    for doc in search_results:
        context += f"section_id: {doc['section_id']}\nsection_title: {doc['section_title']}\npage_number: {doc['page_number']}\ncontent: {doc['content']}\ndocument_id: {doc['document_id']}\ntitle: {doc['title']}\n\n"

# Format the prompt
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [166]:
def llm(prompt):
    # Initialize Groq client
    client = Groq(api_key="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")

# Get the answer from the LLM model
    chat_completion = client.chat.completions.create(
        messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="llama3-8b-8192",
    )
    llm_answer=chat_completion.choices[0].message.content
# return the reslut
    return llm_answer

In [170]:
def rag(query):
    index = process_json_files_and_index(json_folder_path)
    search_results=search(query)
    prompt=build_prompt(query,search_results)
    answer=llm(prompt)
    return(answer)

In [172]:
rag('what is the impact of foring exchange directive in Ethiopian banking industry?')

'Based on the context, the impact of the foreign exchange directive in the Ethiopian banking industry is not explicitly mentioned. However, we can infer that the directive aims to regulate the foreign exchange market and ensure the efficient and transparent operation of foreign exchange transactions.\n\nThe directive establishes an inter-bank foreign exchange market, which allows banks to buy and sell foreign currencies among themselves at freely negotiated rates. This may improve liquidity in the foreign exchange market, making it easier for banks to facilitate foreign exchange transactions.\n\nAdditionally, the directive requires banks to report their exchange rates to the NBE and to maintain Foreign Exchange Retention Accounts for exporters. This may help to improve transparency in the foreign exchange market and ensure that exporters are able to retain a portion of their foreign exchange earnings.\n\nOverall, the foreign exchange directive aims to promote a more orderly and efficie